# Import Dependencies

In [2]:
import gym
from gym import Env
from gym.spaces import Box, Dict, Discrete, MultiDiscrete, MultiBinary, Tuple

import numpy as np
import random 
import os

from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# Types oF Spaces


In [5]:
Discrete(3).sample()

1

In [10]:
Box(0,1,shape=(3,3)).sample() #start value end value and shape consist of box
#we use this if we have continuous varibles or data from sensors 

array([[0.8047057 , 0.1907872 , 0.641977  ],
       [0.8350541 , 0.75820446, 0.3179056 ],
       [0.81854767, 0.77423924, 0.37324792]], dtype=float32)

In [11]:
Tuple((Discrete(3), Box(0,1, shape=(3,1)))).sample()

(1, array([[0.49510986],
        [0.95162815],
        [0.4622848 ]], dtype=float32))

In [12]:
Dict({'height':Discrete(2),'Speed':Box(0,100,shape=(1,)), 'color':MultiBinary(4)}).sample()

OrderedDict([('Speed', array([86.69599], dtype=float32)),
             ('color', array([1, 1, 1, 0], dtype=int8)),
             ('height', 1)])

In [14]:
MultiBinary(4).sample()

array([1, 0, 0, 1], dtype=int8)

In [15]:
MultiDiscrete([5,2,2,5]).sample()

array([4, 0, 1, 1], dtype=int64)

# Building an Environment
- Build an agent to give us the best shower possible
- Randomly temperature
- 37 and 39 degree

In [17]:
Box(low=np.array([0]), high=np.array([100])).sample()

C:\Users\hp\AppData\Local\Programs\Python\Python37\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


array([25.385439], dtype=float32)

In [20]:
Box(0,100,shape=(1,)).sample()

array([37.121197], dtype=float32)

In [22]:
val = Discrete(3).sample()
print(val-1, val)# if 2 then2-1 =1 add temperature if 0 then 0-1 = -1 subtract temperature

0 1


In [44]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100])) #or we can write Box(0,100,shape(1,))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60 #this will be number of times one episode runs
    def step(self, action):#we pass action here and action will do something
        #Apply temperature adjustment 
        self.state += action -1

        #decrease shower time or episode length
        self.shower_length -= 1

        #calculate reward
        if self.state >= 37 and self.state <=39:
            reward = 1
        else:
            reward = -1
        
        if self.shower_length <=0:
            done = True
        else:
            done = False
        info = {}

        return self.state , reward, done ,info
    def render(self):#for printing
        pass
    def reset(self):
        self.state =  np.array([38 + random.randint(-3,3)]).astype(float) #initialize state to random temperature
        self.shower_length = 60 #set length of episode to 60 again
        return self.state

In [41]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [45]:
env = ShowerEnv()

In [34]:
env.observation_space.sample()

array([86.80924], dtype=float32)

In [36]:
env.action_space.sample()

2

In [46]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-46
Episode:2 Score:-60
Episode:3 Score:-36
Episode:4 Score:0
Episode:5 Score:-56


# Training model

In [47]:
log_path = os.path.join('Training', 'Logs')
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)
model.learn(total_timesteps=400000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\hp\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hp\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hp\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hp\AppData\Local\Pr

Logging to Training\Logs\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -34.2    |
| time/              |          |
|    fps             | 313      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -33.1       |
| time/                   |             |
|    fps                  | 423         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008830579 |
|    clip_fraction        | 0.0383      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -2.16e-05   |

In [48]:
save_path = os.path.join("Training" , "Saved Models", "Shower_Model_PPO")
model.save(save_path)

C:\Users\hp\AppData\Local\Programs\Python\Python37\lib\site-packages\stable_baselines3\common\save_util.py:276: UserWarning: Path 'Training\Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [49]:
del model

In [50]:
model = PPO.load(save_path, env)


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [52]:
evaluate_policy(model,env,n_eval_episodes=10)

(12.0, 58.787753826796276)